In [ ]:
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SimpleImputer


In [83]:
train_data = pd.read_csv("../data/train.csv")

print(train_data.describe())

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   456.098091  ..

In [84]:
print(train_data.head(3))

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  

[3 rows x 81 columns]


In [85]:
print(f"Number of empty rows before deleting them in target= {train_data["SalePrice"].isna().sum()}")

train_data.dropna(axis=0, subset=["SalePrice"], inplace=True)

print(f"Number of empty rows after deleting them in target= {train_data["SalePrice"].isna().sum()}")



Number of empty rows before deleting them in target= 0
Number of empty rows after deleting them in target= 0


In [86]:
y = train_data["SalePrice"]
y


0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [87]:
X = train_data.drop("SalePrice", axis=1)
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [88]:
null_count=X.isnull().sum()
total_rows = len(X)
null_percentage = (null_count/total_rows) * 100

null_summary = pd.DataFrame({"Null_values": null_count,
                             "Null_percentage": null_percentage.round(2)
})

columns_with_nulls = null_summary[null_summary["Null_values"]>0]
print(columns_with_nulls.sort_values("Null_percentage",ascending=True))

              Null_values  Null_percentage
Electrical              1             0.07
MasVnrArea              8             0.55
BsmtCond               37             2.53
BsmtFinType1           37             2.53
BsmtQual               37             2.53
BsmtExposure           38             2.60
BsmtFinType2           38             2.60
GarageType             81             5.55
GarageQual             81             5.55
GarageFinish           81             5.55
GarageCond             81             5.55
GarageYrBlt            81             5.55
LotFrontage           259            17.74
FireplaceQu           690            47.26
MasVnrType            872            59.73
Fence                1179            80.75
Alley                1369            93.77
MiscFeature          1406            96.30
PoolQC               1453            99.52


In [89]:
X.drop(["Fence","Alley","MiscFeature","PoolQC"],axis=1, inplace=True)
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,61,0,0,0,0,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,42,0,0,0,0,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,0,112,0,0,0,0,4,2010,WD,Normal


In [90]:
var_num = [col for col in X.columns if X[col].dtype in ["int64","float64"]]
var_num

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [91]:
var_cat = [col for col in X.columns if X[col].dtype == "object" and X[col].nunique() < 10]
var_cat

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [92]:
var_high_corr = [col for col in var_num if X[col].corr(y) > 0.6]
var_high_corr

['OverallQual',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'GarageCars',
 'GarageArea']

In [93]:
var_intermediate_high_corr = [col for col in var_num if X[col].corr(y) > 0.4 and X[col].corr(y) < 0.6]
var_intermediate_high_corr

['YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'FullBath',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt']

In [94]:
var_intermediate_corr = [col for col in var_num if X[col].corr(y) > 0.2 and X[col].corr(y) < 0.4]
var_intermediate_corr

['LotFrontage',
 'LotArea',
 'BsmtFinSF1',
 'BsmtUnfSF',
 '2ndFlrSF',
 'BsmtFullBath',
 'HalfBath',
 'WoodDeckSF',
 'OpenPorchSF']

In [95]:
results = []
for col in var_cat:
    means = train_data.groupby(col)["SalePrice"].mean()
    diff = means.max() - means.min()
    results.append((col, diff))
    print(f"{col}: mean difference = {diff:.2f}")

MSZoning: mean difference = 139486.06
Street: mean difference = 50940.04
LotShape: mean difference = 75078.55
LandContour: mean difference = 88429.86
Utilities: mean difference = 43450.96
LotConfig: mean difference = 46916.57
LandSlope: mean difference = 24422.43
Condition1: mean difference = 90783.33
Condition2: mean difference = 228250.00
BldgType: mean difference = 57331.55
HouseStyle: mean difference = 109850.00
RoofStyle: mean difference = 76090.91
RoofMatl: mean difference = 253250.00
MasVnrType: mean difference = 119265.56
ExterQual: mean difference = 279375.75
ExterCond: mean difference = 124833.33
Foundation: mean difference = 117864.82
BsmtQual: mean difference = 211349.01
BsmtCond: mean difference = 149599.91
BsmtExposure: mean difference = 92037.51
BsmtFinType1: mean difference = 88524.47
BsmtFinType2: mean difference = 58841.11
Heating: mean difference = 109521.20
HeatingQC: mean difference = 127914.43
CentralAir: mean difference = 80922.64
Electrical: mean difference = 11

In [96]:
results_sorted = sorted(results, key=lambda x: x[1])
print("\nOrdenado ascendente:")
for col, diff in results_sorted:
    print(f"{col}: {diff:.2f}")


Ordenado ascendente:
LandSlope: 24422.43
Utilities: 43450.96
LotConfig: 46916.57
Street: 50940.04
BldgType: 57331.55
BsmtFinType2: 58841.11
PavedDrive: 71394.85
LotShape: 75078.55
RoofStyle: 76090.91
GarageCond: 79385.74
CentralAir: 80922.64
LandContour: 88429.86
BsmtFinType1: 88524.47
Condition1: 90783.33
BsmtExposure: 92037.51
Functional: 97629.15
GarageFinish: 97896.27
Heating: 109521.20
HouseStyle: 109850.00
Foundation: 117864.82
MasVnrType: 119265.56
Electrical: 119825.11
ExterCond: 124833.33
HeatingQC: 127914.43
MSZoning: 139486.06
GarageQual: 140833.33
GarageType: 144789.63
BsmtCond: 149599.91
SaleType: 155095.42
SaleCondition: 168166.75
FireplaceQu: 207948.35
BsmtQual: 211349.01
KitchenQual: 222989.46
Condition2: 228250.00
RoofMatl: 253250.00
ExterQual: 279375.75


In [97]:
cat_selected = [col for col, dif in results_sorted if dif > 100000]
cat_selected

['Heating',
 'HouseStyle',
 'Foundation',
 'MasVnrType',
 'Electrical',
 'ExterCond',
 'HeatingQC',
 'MSZoning',
 'GarageQual',
 'GarageType',
 'BsmtCond',
 'SaleType',
 'SaleCondition',
 'FireplaceQu',
 'BsmtQual',
 'KitchenQual',
 'Condition2',
 'RoofMatl',
 'ExterQual']

In [102]:
X_data = var_high_corr + var_intermediate_high_corr + cat_selected
X = X[X_data]
X.head(3)

,OverallQual,TotalBsmtSF,1stFlrSF,GrLivArea,GarageCars,GarageArea,YearBuilt,YearRemodAdd,MasVnrArea,FullBath,...,GarageType,BsmtCond,SaleType,SaleCondition,FireplaceQu,BsmtQual,KitchenQual,Condition2,RoofMatl,ExterQual
0,7,856,856,1710,2,548,2003,2003,196.0,2,...,Attchd,TA,WD,Normal,NaN,Gd,Gd,Norm,CompShg,Gd
1,6,1262,1262,1262,2,460,1976,1976,0.0,2,...,Attchd,TA,WD,Normal,TA,Gd,TA,Norm,CompShg,TA
2,7,920,920,1786,2,608,2001,2002,162.0,2,...,Attchd,TA,WD,Normal,TA,Gd,Gd,Norm,CompShg,Gd
